In [1]:
import os
import torch
import torchvision
import numpy as np
import pandas as pd
from PIL import Image
from torch import optim
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

%matplotlib inline

In [2]:
DATASET_PATH = "./task/"
df = pd.read_csv(os.path.join(DATASET_PATH, "images_labelling.csv"), error_bad_lines=False)

In [3]:
problems = []
for idx, line in df.iterrows():
    if not os.path.exists(os.path.join(DATASET_PATH, 'images', str(line.boxid)+'.png')):
        print(idx)
        problems.append(idx)
df.drop(df.index[problems], inplace=True)

In [4]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [5]:
train_df.head()

,boxid,label,class_
2014,60771,16,A19
2238,313,6,B22
485,24415,15,A15
2177,53926,6,B22
3534,63193,7,A8


In [6]:
class PlayersDataset(Dataset):
    
    class__list = df['class_'].unique()
    class2num = {class_:i for i, class_ in enumerate(class__list)}
    num2class = {i:class_ for i, class_ in enumerate(class__list)}
    def __init__(self, root, dataframe, transform=None):
        super(PlayersDataset, self).__init__()
        self.dataframe = dataframe
        self.root = root
        if transform is None:
            transform = torchvision.transforms.Compose([
                torchvision.transforms.Resize((224, 224)),
                torchvision.transforms.ToTensor()
            ])
        self.transform = transform
        
    def __getitem__(self, idx):
        line = self.dataframe.iloc[idx]
        class_ = line.class_
        class__id = self.class2num[class_]
        img_path = os.path.join(self.root, str(line.boxid)+'.png')
        img = Image.open(img_path).convert('RGB')
        img_tensor = self.transform(img)
        return img_tensor, class__id
            
    def __len__(self):
        return len(self.dataframe)

In [7]:
# Construct dataset and dataloader
train_ds = PlayersDataset(os.path.join(DATASET_PATH, 'images'), train_df)
test_ds = PlayersDataset(os.path.join(DATASET_PATH, 'images'), test_df)
train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=16, shuffle=False)

In [8]:
from torchvision.models import ResNet, resnet18
import torch.nn as nn

In [9]:
class PlayersModel(nn.Module):
  
    def __init__(self, num_classes):
        super(PlayersModel, self).__init__()
        backbone = resnet18(pretrained=True)
        backbone.fc = nn.Linear(backbone.fc.in_features, num_classes)
        self.backbone = backbone
    def forward(self, inp):
        return self.backbone(inp)

In [10]:
model = PlayersModel(len(df.class_.unique()))

In [11]:
def train_val_loop(model, save_path):
    epochs = 50
    log_step = 20
    optimizer = optim.RMSprop(model.parameters(), lr=0.01)
    criterion = nn.CrossEntropyLoss()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    model.to(device)
    loss_hist = []
    for epoch in range(1, epochs+1):
        # train phase
        model.train()
        for bth_num, batch in enumerate(train_loader, 1):
            images, labels = batch
            images, labels = images.to(device), labels.to(device)
            logits = model(images)
            loss = criterion(logits, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if bth_num % log_step == 0:
                print("\r Epoch: {}, # {} => Loss: {:.4f}".format(epoch, bth_num, loss.item()), flush=True, end='')
                loss_hist.append(loss.item())
        print()
        # Valid phase
        model.eval()
        total_preds = []
        total_labels = []
        for bth_num, batch in enumerate(test_loader, 1):
            images, labels = batch
            images, labels = images.to(device), labels.to(device)
            logits = model(images)
            preds = logits.argmax(dim=-1)
            total_labels.append(labels.cpu().data.numpy())
            total_preds.append(preds.cpu().data.numpy())
        total_preds = np.concatenate(total_preds)
        total_labels = np.concatenate(total_labels)
        acc = sum(total_preds == total_labels) / len(total_labels)
        print("Accuracy: {:.2%}".format(acc))
    
    # Save model weights
    torch.save(model.state_dict(), save_path)
    
    return loss_hist

In [12]:
loss_hist = train_val_loop(model, 'resnet18-2.pth')

 Epoch: 1, # 180 => Loss: 2.2692
Accuracy: 16.80%
 Epoch: 2, # 180 => Loss: 1.6147
Accuracy: 28.53%
 Epoch: 3, # 180 => Loss: 2.0019
Accuracy: 19.60%
 Epoch: 4, # 180 => Loss: 0.9418
Accuracy: 44.93%
 Epoch: 5, # 180 => Loss: 0.9915
Accuracy: 61.20%
 Epoch: 6, # 180 => Loss: 0.6716
Accuracy: 51.20%
 Epoch: 7, # 180 => Loss: 0.7695
Accuracy: 50.80%
 Epoch: 8, # 180 => Loss: 0.2408
Accuracy: 68.67%
 Epoch: 9, # 180 => Loss: 0.2827
Accuracy: 74.00%
 Epoch: 10, # 180 => Loss: 0.5131
Accuracy: 60.40%
 Epoch: 11, # 180 => Loss: 0.1255
Accuracy: 69.47%
 Epoch: 12, # 180 => Loss: 0.7373
Accuracy: 82.53%
 Epoch: 13, # 180 => Loss: 0.2598
Accuracy: 57.73%
 Epoch: 14, # 180 => Loss: 0.5910
Accuracy: 76.67%
 Epoch: 15, # 180 => Loss: 0.5311
Accuracy: 83.07%
 Epoch: 16, # 180 => Loss: 0.1148
Accuracy: 90.67%
 Epoch: 17, # 180 => Loss: 0.1911
Accuracy: 52.80%
 Epoch: 18, # 180 => Loss: 0.0513
Accuracy: 88.40%
 Epoch: 19, # 180 => Loss: 0.0315
Accuracy: 89.73%
 Epoch: 20, # 180 => Loss: 0.3398
Accura